In [1]:
import numpy as np
import librosa
import os
import time

In [2]:
def fft_extract(filename):
  y, sr = librosa.load(filename, sr=16000)
  fft = np.abs(np.fft.fft(y))
  fft_feature = np.ndarray.flatten(fft)
  return fft_feature

In [3]:
def window_slicing(array, window_size, number_of_windows, overlap):
  windows = []
  for i in range(0, number_of_windows):
    start = i * window_size - overlap
    end = start + window_size
    if end > len(array):
      end = len(array)
    windows.append(array[start:end])
  return windows

In [70]:
def feature_extract(windows):
  features = []
  for window in windows:
    means = []
    stds = []
    for i in range(0, len(window)):
      mean = np.mean(window[i])
      std = np.std(window[i])
      means.append(mean)
      stds.append(std)
    features.append(means)
    features.append(stds)
  return features

In [71]:
def get_feature(filename):
    fft_npa = fft_extract(filename)
    fft_ws = window_slicing(fft_npa, window_size = 500, number_of_windows = 64)
    fft_feature = feature_extract(fft_ws)
    final_feature = np.array(fft_feature)
    final_feature = final_feature.reshape(final_feature.shape[0], final_feature.shape[1] * final_feature.shape[2])
    return final_feature

In [57]:
def extract_features_from_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    all_fft_features = []

    wav_files = [f for f in os.listdir(input_folder) if f.endswith('.wav')]

    for wav_file in wav_files:
        filename = os.path.join(input_folder, wav_file)
        fft_features = get_feature(filename)
        all_fft_features.append(fft_features)

    combined_save_path = os.path.join(output_folder, f'{os.path.basename(input_folder)}_fft.npy')
    np.save(combined_save_path, np.array(all_fft_features))

    return combined_save_path, len(all_fft_features)

In [16]:
root_folder = 'data'

subfolders = ['train', 'test', 'val']
categories = ['Queen', 'Queenless']

start_time = time.time()

output_paths = []
total_features = 0

for subfolder in subfolders:
    for category in categories:
        input_folder = os.path.join(root_folder, subfolder, category)
        output_folder = os.path.join('new_fft_npa11', subfolder, f'{category}_FFT')
        output_path, num_features = extract_features_from_folder(input_folder, output_folder)
        output_paths.append(output_path)
        total_features += num_features

end_time = time.time()
total_time = end_time - start_time

In [17]:
for path in output_paths:
    print(f'Features saved at: {path}')

print(f'Total features extracted using FFT: {total_features}')
print(f'Feature extraction completed in {total_time:.2f} seconds')

Features saved at: new_fft_npa11\train\Queen_FFT\Queen_fft.npy
Features saved at: new_fft_npa11\train\Queenless_FFT\Queenless_fft.npy
Features saved at: new_fft_npa11\test\Queen_FFT\Queen_fft.npy
Features saved at: new_fft_npa11\test\Queenless_FFT\Queenless_fft.npy
Features saved at: new_fft_npa11\val\Queen_FFT\Queen_fft.npy
Features saved at: new_fft_npa11\val\Queenless_FFT\Queenless_fft.npy
Total features extracted using FFT: 20000
Feature extraction completed in 165.66 seconds
